In [6]:
import os
from tqdm import tqdm
import pandas as pd
from ensemble_boxes import *

In [7]:
DATA_DIR = '../data/csv'
CSV_FILE = 'test.csv'
DOWN_SIZE = 2

In [8]:
train = pd.read_csv(os.path.join(DATA_DIR, CSV_FILE))

In [61]:
for idx, row in tqdm(train.iterrows(), total=len(train), position=0, leave=True):
    if row['class_id'] == 14:
        train.at[idx, 'x_min'] = 0
        train.at[idx, 'y_min'] = 0
        train.at[idx, 'x_max'] = row['width']
        train.at[idx, 'y_max'] = row['height']

100%|██████████| 67914/67914 [00:10<00:00, 6395.29it/s]


In [62]:
train[['x_min', 'y_min', 'x_max', 'y_max', 'width', 'height']] = \
train[['x_min', 'y_min', 'x_max', 'y_max', 'width', 'height']] / DOWN_SIZE

In [63]:
print(train.shape)
train.head()

(67914, 10)


,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,0.0,0.0,1166.0,1290.0,1166.0,1290.0
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,0.0,0.0,1477.0,1579.5,1477.0,1579.5
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,345.5,687.5,826.5,915.5,1040.0,1168.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,632.0,371.5,805.5,509.5,1152.0,1440.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,0.0,0.0,1270.0,1536.0,1270.0,1536.0


In [64]:
train['class_id'].value_counts()

14    31818
0      7162
3      5427
11     4842
13     4655
8      2580
7      2483
10     2476
9      2203
6      1247
5      1000
2       960
4       556
1       279
12      226
Name: class_id, dtype: int64

In [65]:
# ===============================
# Default WBF config (you can change these)
iou_thr = 0.5
skip_box_thr = 0.0001
sigma = 0.1
# ===============================


def preprocess_fusion(df, fusion_type, iou_thr=0.5, sigma=0.1, skip_box_thr=0.0001):
    # Loading the train DF
#     df.fillna(0, inplace=True)
#     df.loc[df["class_id"] == 14, ['x_max', 'y_max']] = 1.0

    results = []
    image_ids = df["image_id"].unique()

    for image_id in tqdm(image_ids, total=len(image_ids), position=0, leave=True):

        # All annotations for the current image.
        data = df[df["image_id"] == image_id]
        data = data.reset_index(drop=True)
        width = data.loc[0, 'width']
        height = data.loc[0, 'height']

        annotations = {}
        weights = []

        # WBF expects the coordinates in 0-1 range.
        max_value = data.iloc[:, 4:].values.max()
        data.loc[:, ["x_min", "y_min", "x_max", "y_max"]] = data.iloc[:, 4:8] / max_value

        # Loop through all of the annotations for single image
        for idx, row in data.iterrows():

            rad_id = row["rad_id"]

            if rad_id not in annotations:
                annotations[rad_id] = {
                    "boxes_list": [],
                    "scores_list": [],
                    "labels_list": [],
                }

                # We consider all of the radiologists as equal.
                weights.append(1.0)

            annotations[rad_id]["boxes_list"].append([row["x_min"], row["y_min"], row["x_max"], row["y_max"]])
            annotations[rad_id]["scores_list"].append(1.0)
            annotations[rad_id]["labels_list"].append(row["class_id"])

        boxes_list = []
        scores_list = []
        labels_list = []

        for annotator in annotations.keys():
            boxes_list.append(annotations[annotator]["boxes_list"])
            scores_list.append(annotations[annotator]["scores_list"])
            labels_list.append(annotations[annotator]["labels_list"])

        # Calculate Fusion
        if fusion_type == 'wbf':
            boxes, scores, labels = weighted_boxes_fusion(
                boxes_list,
                scores_list,
                labels_list,
                weights=weights,
                iou_thr=iou_thr,
                skip_box_thr=skip_box_thr)
            
        if fusion_type == 'nms':
             boxes, scores, labels = nms(
                boxes_list,
                scores_list,
                labels_list,
                weights=weights,
                iou_thr=iou_thr)
                
        if fusion_type == 'softnms':
            boxes, scores, labels = soft_nms(
                boxes_list,
                scores_list,
                labels_list,
                sigma=sigma,
                weights=weights,
                iou_thr=iou_thr)
                
        if fusion_type == 'nmw':
            boxes, scores, labels = non_maximum_weighted(
                boxes_list,
                scores_list,
                labels_list,
                weights=weights,
                iou_thr=iou_thr,
                skip_box_thr=skip_box_thr)    
                
                       

        for idx, box in enumerate(boxes):
            results.append({
                "image_id": image_id,
                "class_id": int(labels[idx]),
                "rad_id": "wbf",
                "x_min": box[0] * max_value,
                "y_min": box[1] * max_value,
                "x_max": box[2] * max_value,
                "y_max": box[3] * max_value,
                "height": height,
                "width": width
            })
            
    results = pd.DataFrame(results, columns=['image_id','class_id','rad_id','x_min','y_min','x_max','y_max','width','height'])
    return results

In [66]:
wbf_csv = preprocess_fusion(train, 'wbf')

100%|██████████| 15000/15000 [02:50<00:00, 88.13it/s]


In [67]:
print(wbf_csv.shape)
wbf_csv.head()

(34510, 9)


,image_id,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
0,50a418190bc3fb1ef1633bf9678929b3,14,wbf,0.000000,0.000000,1166.000049,1290.000000,1166.0,1290.0
1,21a10246a5ec7af151081d0cd6d65dc9,14,wbf,0.000000,0.000000,1476.999989,1579.500000,1477.0,1579.5
2,9a5094b2563a1ef3ff50dc5c7ff71345,3,wbf,345.333339,677.166659,829.333302,898.833385,1040.0,1168.0
3,9a5094b2563a1ef3ff50dc5c7ff71345,0,wbf,526.000000,357.500000,649.500000,483.000000,1040.0,1168.0
4,9a5094b2563a1ef3ff50dc5c7ff71345,11,wbf,894.500000,864.500000,937.500000,996.000000,1040.0,1168.0


In [45]:
#normalise bbox
# wbf_csv['x_min'] = wbf_csv['x_min'] / wbf_csv['width']
# wbf_csv['x_max'] = wbf_csv['x_max'] / wbf_csv['width']
# wbf_csv['y_min'] = wbf_csv['y_min'] / wbf_csv['height']
# wbf_csv['y_max'] = wbf_csv['y_max'] / wbf_csv['height']

In [68]:
# split folds
from sklearn.model_selection import GroupKFold

wbf_csv['fold'] = -1
gkf  = GroupKFold(n_splits = 5)
for fold, (train_idx, val_idx) in enumerate(gkf.split(wbf_csv, groups=wbf_csv.image_id.tolist())):
    wbf_csv.loc[val_idx, 'fold'] = fold

wbf_csv.head()

,image_id,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,fold
0,50a418190bc3fb1ef1633bf9678929b3,14,wbf,0.000000,0.000000,1166.000049,1290.000000,1166.0,1290.0,4
1,21a10246a5ec7af151081d0cd6d65dc9,14,wbf,0.000000,0.000000,1476.999989,1579.500000,1477.0,1579.5,2
2,9a5094b2563a1ef3ff50dc5c7ff71345,3,wbf,345.333339,677.166659,829.333302,898.833385,1040.0,1168.0,3
3,9a5094b2563a1ef3ff50dc5c7ff71345,0,wbf,526.000000,357.500000,649.500000,483.000000,1040.0,1168.0,3
4,9a5094b2563a1ef3ff50dc5c7ff71345,11,wbf,894.500000,864.500000,937.500000,996.000000,1040.0,1168.0,3


In [69]:
wbf_csv.groupby(['fold'])['class_id'].value_counts()

fold  class_id
0     14          2127
      11           784
      0            680
      13           657
      3            493
      7            424
      8            385
      9            373
      10           326
      6            198
      5            164
      2            150
      4             79
      1             40
      12            22
1     14          2118
      11           829
      0            707
      13           647
      3            500
      10           379
      9            378
      7            364
      8            359
      2            179
      6            158
      5            140
      4             75
      1             44
      12            25
                  ... 
3     14          2133
      11           792
      13           723
      0            661
      3            470
      7            408
      8            389
      9            357
      10           335
      6            214
      2            139
      5            

In [70]:
wbf_csv.to_csv(os.path.join(DATA_DIR, 'train_downsampled_fold.csv'), index=False)